<a href="https://colab.research.google.com/github/diegoesquivel09/ejerciciosbsm/blob/main/Eurosourcing_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mip

from mip import *


### Definir objeto modelo y los datos


In [2]:
#Crear el modelo

model = Model()

# Datos del problema. --

mercados = ['Madrid', 'París', 'Berlín', 'Roma', 'Amsterdam']
costo_produccion_local = {'Madrid': 460, 'París': 515, 'Berlín': 505, 'Roma': 480, 'Amsterdam': 490}
demanda = {'Madrid': 420, 'París': 1310, 'Berlín': 1250, 'Roma': 930, 'Amsterdam': 870}
#Agregar costo transporte terrestre 0 entre iguales ciudades
costo_transporte_terrestre = {
    ('Madrid', 'París'): 10, ('Madrid', 'Berlín'): 35, ('Madrid', 'Roma'): 25, ('Madrid', 'Amsterdam'): 25, ('Madrid', 'Madrid'): 0,
    ('París', 'Berlín'): 15, ('París', 'Roma'): 20, ('París', 'Amsterdam'): 10, ('París', 'Madrid'): 10, ('París', 'París'): 0,
    ('Berlín', 'Madrid'): 35,('Berlín', 'París'): 15,('Berlín', 'Roma'): 25, ('Berlín', 'Amsterdam'): 10, ('Berlín', 'Berlín'): 0,
    ('Roma', 'Madrid'): 25, ('Roma', 'París'): 20, ('Roma', 'Berlín'): 25, ('Roma', 'Amsterdam'): 23, ('Roma', 'Roma'): 0,
    ('Amsterdam', 'Madrid'): 25, ('Amsterdam', 'París'): 10, ('Amsterdam', 'Berlín'): 10, ('Amsterdam', 'Roma'): 23, ('Amsterdam', 'Amsterdam'): 0,
}
costo_produccion_china_40 = {'Madrid': (1800+440*200), 'París': (1650+440*200), 'Berlín': (1800+440*200), 'Roma': (2000+440*200), 'Amsterdam': (1200+440*200)}#Costo producción en China + transporte container 40
costo_produccion_china_20 = {'Madrid': (1100+440*100), 'París': (1050+440*100), 'Berlín': (1100+440*100), 'Roma': (1200+440*100), 'Amsterdam': (700+440*100)}#Costo producción en China + transporte container 20
costo_produccion_china_LCL = {'Madrid': (23+440), 'París': (22+440), 'Berlín': (23+440), 'Roma': (25+440), 'Amsterdam': (15+440)}#Costo producción en China + transporte LCL

print(costo_produccion_china_LCL)

{'Madrid': 463, 'París': 462, 'Berlín': 463, 'Roma': 465, 'Amsterdam': 455}


#Definir variables

In [3]:
### Definir variables de decisión
comprar_china_container40= {m: model.add_var(var_type='I', lb=0) for m in mercados} ## Producos OVIS a cada mercado/almacen en container grande
comprar_china_container20 = {m: model.add_var(var_type='I', lb=0) for m in mercados} ## Producos OVIS a cada mercado/almacen en container mediano
comprar_china_LCL = {m: model.add_var(var_type='I', lb=0) for m in mercados} ## Producos OVIS a cada mercado/almacen individualmente
comprar_local = {m: model.add_var(var_type='I', lb=0) for m in mercados} ## Productos producidos en DOMES en un mercado
envios = {(o, d): model.add_var(var_type='I', lb=0) for o in mercados for d in mercados} ## Envíos desde un almacen a otro

produce_local = {m: model.add_var(var_type=BINARY) for m in mercados} ##


#Restricciones de demanda

In [4]:
for m in mercados:
    model.add_constr(comprar_china_LCL[m] + 100*comprar_china_container20[m] + 200*comprar_china_container40[m] + comprar_local[m] + xsum(envios[(o, m)] for o in mercados) == demanda[m] + xsum(envios[(m, d)] for d in mercados))
    model.add_constr(xsum(produce_local[m] for m in mercados) == 1) ## Solo se puede producir en un solo lugar de DOMES
    model.add_constr (comprar_local[m] <= produce_local[m] * 5000 ) #asociar produce local con compra local

model.add_constr (comprar_local[m] >= 0) #verificar el mínimo que se produce internamente


# Función Objetivo

In [5]:
model.objective= minimize (
    xsum(comprar_china_LCL[m] * costo_produccion_china_LCL[m] for m in mercados) +
    xsum(comprar_china_container20[m] * costo_produccion_china_20[m] for m in mercados) +
    xsum(comprar_china_container40[m] * costo_produccion_china_40[m] for m in mercados) +
    xsum(comprar_local[m] * costo_produccion_local[m] for m in mercados) +
    xsum(envios[o, d] * costo_transporte_terrestre[o,d] for o in mercados for d in mercados) ##si toma igual ciudad para origen y destino debería tomar valor 0
)



Resolver el model

In [6]:
status= model.optimize()
print("Estado:", status)

print("Cantidad a comprar desde China:")
for m in mercados:
    print(f"{m}: {comprar_china_container40[m].x}", "containers de 40'")
    print(f"{m}: {comprar_china_container20[m].x}", "containers de 20'")
    print(f"{m}: {comprar_china_LCL[m].x}", "piezas individuales'")


print("\nCantidad a comprar localmente:")
for m in mercados:
    print(f"{m}: {comprar_local[m].x}")

print("Envíos:")
for o in mercados:
    for d in mercados:
        print(f"De {o} a {d}: {envios[o, d].x}")

print("Costo total de producción y transporte:",  model.objective_value)

Estado: OptimizationStatus.OPTIMAL
Cantidad a comprar desde China:
Madrid: 2.0 containers de 40'
Madrid: 0.0 containers de 20'
Madrid: 0.0 piezas individuales'
París: 6.0 containers de 40'
París: 1.0 containers de 20'
París: 0.0 piezas individuales'
Berlín: 6.0 containers de 40'
Berlín: 0.0 containers de 20'
Berlín: 0.0 piezas individuales'
Roma: 4.0 containers de 40'
Roma: 1.0 containers de 20'
Roma: 30.0 piezas individuales'
Amsterdam: 4.0 containers de 40'
Amsterdam: 1.0 containers de 20'
Amsterdam: 30.0 piezas individuales'

Cantidad a comprar localmente:
Madrid: 20.0
París: 0.0
Berlín: 0.0
Roma: 0.0
Amsterdam: 0.0
Envíos:
De Madrid a Madrid: 0.0
De Madrid a París: 0.0
De Madrid a Berlín: 0.0
De Madrid a Roma: 0.0
De Madrid a Amsterdam: 0.0
De París a Madrid: 0.0
De París a París: 0.0
De París a Berlín: 0.0
De París a Roma: 0.0
De París a Amsterdam: 0.0
De Berlín a Madrid: 0.0
De Berlín a París: 0.0
De Berlín a Berlín: 0.0
De Berlín a Roma: 0.0
De Berlín a Amsterdam: 0.0
De Roma a 

In [7]:
total= sum(demanda.values())
print(total)

4780
